In [1]:
import sys
# optionally add scripts location to path
if False:
    sys.path.append("/home/physics3/rroussel/SLAC_Xopt/")
    sys.path.append("/home/physics3/rroussel/SLAC_Xopt/NN_prior/")
    print(sys.path)

In [2]:
sys.path.append("../../../SLAC_Xopt/")
sys.path.append("../../../SLAC_Xopt/NN_prior/")

In [3]:
import time
from typing import Dict

import torch
import numpy as np
import matplotlib.pyplot as plt

from scripts.evaluate_function.screen_image import measure_beamsize, measure_background
from scripts.optimize_function import optimize_function
# from scripts.characterize_emittance import characterize_emittance

from xopt import Xopt, VOCS
from xopt.evaluator import Evaluator
from xopt.numerical_optimizer import LBFGSOptimizer
from xopt.generators import ExpectedImprovementGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from lume_model.utils import variables_from_yaml
from lume_model.torch import LUMEModule, PyTorchModel

from utils import update_variables, create_vocs
from custom_mean import CustomMean, LinearCalibration
from dynamic_custom_mean import DynamicCustomMean
from metric_informed_custom_mean import MetricInformedCustomMean

## Initialization

In [4]:
# set up data saving locations
data_dir = "/home/physics3/ml_tuning/20230729_LCLS_Injector"

run_name = "nn_optimize_1"
run_dir = f"{data_dir}/{run_name}"
import os
# if not os.path.exists(run_dir):
#     os.mkdir(run_dir)

In [5]:
## import variable range
import pandas as pd
filename = "../variables.csv"
VARIABLE_RANGES = pd.read_csv(filename, index_col=0, header=None).T.to_dict(orient='list')
SCREEN_NAME = "OTRS:IN20:621" # OTR 3

In [6]:
# set up background
BACKGROUND_FILE = f"{data_dir}/{SCREEN_NAME}_background.npy".replace(":","_")

## define evaluate function

In [7]:
from epics import caget_many
import json
#with open("../../secondary_variables.json", "r") as f:
#    secondary_observables = json.loads(f)

# define function to measure the total size on OTR4
def eval_beamsize(input_dict):
    results = measure_beamsize(input_dict)

    # get secondary PV settings/readbacks
    # secondary_results = caget_many(secondary_observables)
    # results = results | secondary_results

    results["S_x_mm"] = np.array(results["Sx"]) * 1e-3
    results["S_y_mm"] = np.array(results["Sy"]) * 1e-3

    #add total beam size
    xnp = results["S_x_mm"]
    ynp = results["S_y_mm"]
    results["total_size"] = np.sqrt(xnp**2 + ynp**2)
    return results

## Create model

In [8]:
# model_path = "/home/physics3/rroussel/lcls_cu_injector_nn_model/"
model_path = "../../../lcls_cu_injector_nn_model/"

In [9]:
# load sim_to_nn transformers
input_sim_to_nn = torch.load(model_path + "model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load(model_path + "model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load(model_path + "model/input_pv_to_sim.pt")
output_pv_to_sim = torch.load(model_path + "model/output_pv_to_sim.pt")

In [10]:
# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(open(model_path + "model/pv_variables.yml"))

In [11]:
# create LUME-model
# replace keys in input variables
input_variables = {name.replace("BACT", "BCTRL"): ele for name, ele in input_variables.items()}

lume_model = PyTorchModel(
    model_file=model_path + "model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],  # pv_to_sim before sim_to_nn
    output_transformers=[output_sim_to_nn, output_pv_to_sim],  # sim_to_nn before pv_to_sim
)
print(lume_model.features)

['CAMR:IN20:186:R_DIST', 'Pulse_length', 'FBCK:BCI0:1:CHRG_S', 'SOLN:IN20:121:BCTRL', 'QUAD:IN20:121:BCTRL', 'QUAD:IN20:122:BCTRL', 'ACCL:IN20:300:L0A_ADES', 'ACCL:IN20:300:L0A_PDES', 'ACCL:IN20:400:L0B_ADES', 'ACCL:IN20:400:L0B_PDES', 'QUAD:IN20:361:BCTRL', 'QUAD:IN20:371:BCTRL', 'QUAD:IN20:425:BCTRL', 'QUAD:IN20:441:BCTRL', 'QUAD:IN20:511:BCTRL', 'QUAD:IN20:525:BCTRL']


## Build vocs

In [12]:
# inputs
TUNING_VARIABLES = [
    "SOLN:IN20:121:BCTRL",
    "QUAD:IN20:121:BCTRL",
    "QUAD:IN20:122:BCTRL",
    "QUAD:IN20:361:BCTRL",
    "QUAD:IN20:371:BCTRL",
    "QUAD:IN20:425:BCTRL",
    "QUAD:IN20:441:BCTRL",
    "QUAD:IN20:511:BCTRL",
    "QUAD:IN20:525:BCTRL"
]

ROI = None

measurement_options = {
    "screen": SCREEN_NAME,
    "background": BACKGROUND_FILE,
    "roi": ROI,
    "bb_half_width": 2.0, # half width of the bounding box in terms of std
    "visualize": False,
    "save_img_location": run_dir,
    "sleep_time": 3.0,
    "n_shots": 5
}

image_constraints = {
    "bb_penalty": ["LESS_THAN", 0.0],
    "log10_total_intensity": ["GREATER_THAN", 4]
}

In [13]:
inputs_small = input_pv_to_sim.untransform(torch.load(model_path + "info/inputs_small.pt"))
updated_variables = update_variables(VARIABLE_RANGES, input_variables, inputs_small, from_machine_state=False)
updated_variables

Redefined default value for: QUAD:IN20:361:BCTRL
Redefined default value for: QUAD:IN20:371:BCTRL


{'SOLN:IN20:121:BCTRL': [0.465, 0.48, 0.4779693455075814],
 'QUAD:IN20:121:BCTRL': [-0.015, 0.015, -0.001499227120199691],
 'QUAD:IN20:122:BCTRL': [-0.015, 0.015, -0.0006872989433749197],
 'QUAD:IN20:361:BCTRL': [-3.7, -3.0, -3.0368496452987706],
 'QUAD:IN20:371:BCTRL': [2.448, 2.992, 2.860889851203609],
 'QUAD:IN20:425:BCTRL': [-3.0, 3.0, -0.20217600214382594],
 'QUAD:IN20:441:BCTRL': [-3.0, 3.0, -0.17938799998564897],
 'QUAD:IN20:511:BCTRL': [-3.0, 3.0, 2.852171999771826],
 'QUAD:IN20:525:BCTRL': [-5.0, 5.0, -3.218399988942528]}

In [14]:
vocs = create_vocs(measurement_options, image_constraints, updated_variables, case=1)
print(vocs.as_yaml())

variables:
  SOLN:IN20:121:BCTRL: [0.465, 0.48]
  QUAD:IN20:121:BCTRL: [-0.015, 0.015]
  QUAD:IN20:122:BCTRL: [-0.015, 0.015]
constraints:
  bb_penalty: [LESS_THAN, 0.0]
  log10_total_intensity: [GREATER_THAN, 4.0]
objectives: {total_size: MINIMIZE}
constants: {screen: 'OTRS:IN20:621', background: /home/physics3/ml_tuning/20230729_LCLS_Injector/OTRS_IN20_621_background.npy,
  roi: null, bb_half_width: 2.0, visualize: false, save_img_location: /home/physics3/ml_tuning/20230729_LCLS_Injector/nn_optimize_1,
  sleep_time: 3.0, n_shots: 5, 'QUAD:IN20:361:BCTRL': -3.0368496452987706, 'QUAD:IN20:371:BCTRL': 2.860889851203609,
  'QUAD:IN20:425:BCTRL': -0.20217600214382594, 'QUAD:IN20:441:BCTRL': -0.17938799998564897,
  'QUAD:IN20:511:BCTRL': 2.852171999771826, 'QUAD:IN20:525:BCTRL': -3.218399988942528}
observables: []



## Create Xopt instance

In [15]:
# Xopt definition
model_constructor = StandardModelConstructor()
generator = ExpectedImprovementGenerator(
    vocs=vocs,
    model_constructor=model_constructor,
)
generator.numerical_optimizer.max_iter = 200
evaluator = Evaluator(function=eval_beamsize)
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)

## Create initial samples

In [15]:
# create initial samples
n_init = 3
X.random_evaluate(n_samples=n_init)

CAPUT SOLN:IN20:121:BCTRL 0.47841640986706185
CAPUT QUAD:IN20:121:BCTRL -0.012552011501866633
CAPUT QUAD:IN20:122:BCTRL 0.010030815883403658
CAPUT QUAD:IN20:361:BCTRL -3.68396241469784
CAPUT QUAD:IN20:371:BCTRL 2.7846364518655244
CAPUT QUAD:IN20:425:BCTRL -0.14603797138561614
CAPUT QUAD:IN20:441:BCTRL -2.079346405304932
CAPUT QUAD:IN20:511:BCTRL 1.9016997681407224
CAPUT QUAD:IN20:525:BCTRL 4.672262581879604
CAPUT SOLN:IN20:121:BCTRL 0.472328136148071
CAPUT QUAD:IN20:121:BCTRL -0.00248732783701662
CAPUT QUAD:IN20:122:BCTRL -0.004659970435753139
CAPUT QUAD:IN20:361:BCTRL -3.697688085714131
CAPUT QUAD:IN20:371:BCTRL 2.7071720367521577
CAPUT QUAD:IN20:425:BCTRL 2.080630730543464
CAPUT QUAD:IN20:441:BCTRL 0.18299609295831498
CAPUT QUAD:IN20:511:BCTRL 2.6485888590380524
CAPUT QUAD:IN20:525:BCTRL -0.24911646185996084
CAPUT SOLN:IN20:121:BCTRL 0.4735814293955094
CAPUT QUAD:IN20:121:BCTRL -0.002169547494139768
CAPUT QUAD:IN20:122:BCTRL -0.0034133176824396585
CAPUT QUAD:IN20:361:BCTRL -3.6761368

,SOLN:IN20:121:BCTRL,QUAD:IN20:121:BCTRL,QUAD:IN20:122:BCTRL,QUAD:IN20:361:BCTRL,QUAD:IN20:371:BCTRL,QUAD:IN20:425:BCTRL,QUAD:IN20:441:BCTRL,QUAD:IN20:511:BCTRL,QUAD:IN20:525:BCTRL,screen,...,Sy,bb_penalty,total_intensity,log10_total_intensity,time,S_x_mm,S_y_mm,total_size,xopt_runtime,xopt_error
1,0.478416,-0.012552,0.010031,-3.683962,2.784636,-0.146038,-2.079346,1.901700,4.672263,OTRS:IN20:621,...,"[355.73989908331197, nan, nan, nan, nan]",-151.230709,"[5850.27529337697, 852.7765415482748, 457.9262...",3.767176,"[1690675164.217087, 1690675165.6869926, 169067...","[0.17487612038026987, nan, nan, nan, nan]","[0.355739899083312, nan, nan, nan, nan]",0.396399,10.484435,False
1,0.478416,-0.012552,0.010031,-3.683962,2.784636,-0.146038,-2.079346,1.901700,4.672263,OTRS:IN20:621,...,"[355.73989908331197, nan, nan, nan, nan]",19.797624,"[5850.27529337697, 852.7765415482748, 457.9262...",2.930835,"[1690675164.217087, 1690675165.6869926, 169067...","[0.17487612038026987, nan, nan, nan, nan]","[0.355739899083312, nan, nan, nan, nan]",NaN,10.484435,False
1,0.478416,-0.012552,0.010031,-3.683962,2.784636,-0.146038,-2.079346,1.901700,4.672263,OTRS:IN20:621,...,"[355.73989908331197, nan, nan, nan, nan]",105.014281,"[5850.27529337697, 852.7765415482748, 457.9262...",2.660796,"[1690675164.217087, 1690675165.6869926, 169067...","[0.17487612038026987, nan, nan, nan, nan]","[0.355739899083312, nan, nan, nan, nan]",NaN,10.484435,False
1,0.478416,-0.012552,0.010031,-3.683962,2.784636,-0.146038,-2.079346,1.901700,4.672263,OTRS:IN20:621,...,"[355.73989908331197, nan, nan, nan, nan]",161.697935,"[5850.27529337697, 852.7765415482748, 457.9262...",2.657556,"[1690675164.217087, 1690675165.6869926, 169067...","[0.17487612038026987, nan, nan, nan, nan]","[0.355739899083312, nan, nan, nan, nan]",NaN,10.484435,False
1,0.478416,-0.012552,0.010031,-3.683962,2.784636,-0.146038,-2.079346,1.901700,4.672263,OTRS:IN20:621,...,"[355.73989908331197, nan, nan, nan, nan]",72.861204,"[5850.27529337697, 852.7765415482748, 457.9262...",2.785223,"[1690675164.217087, 1690675165.6869926, 169067...","[0.17487612038026987, nan, nan, nan, nan]","[0.355739899083312, nan, nan, nan, nan]",NaN,10.484435,False
2,0.472328,-0.002487,-0.004660,-3.697688,2.707172,2.080631,0.182996,2.648589,-0.249116,OTRS:IN20:621,...,"[nan, 621.4792467640008, nan, nan, nan]",60.985551,"[1790.4556854108073, 1406.4239875995156, 1466....",3.252964,"[1690675174.3083148, 1690675175.7930713, 16906...","[nan, 0.3657731398755853, nan, nan, nan]","[nan, 0.6214792467640008, nan, nan, nan]",NaN,10.125762,False
2,0.472328,-0.002487,-0.004660,-3.697688,2.707172,2.080631,0.182996,2.648589,-0.249116,OTRS:IN20:621,...,"[nan, 621.4792467640008, nan, nan, nan]",-76.483872,"[1790.4556854108073, 1406.4239875995156, 1466....",3.148116,"[1690675174.3083148, 1690675175.7930713, 16906...","[nan, 0.3657731398755853, nan, nan, nan]","[nan, 0.6214792467640008, nan, nan, nan]",0.721129,10.125762,False
2,0.472328,-0.002487,-0.004660,-3.697688,2.707172,2.080631,0.182996,2.648589,-0.249116,OTRS:IN20:621,...,"[nan, 621.4792467640008, nan, nan, nan]",13.059391,"[1790.4556854108073, 1406.4239875995156, 1466....",3.166358,"[1690675174.3083148, 1690675175.7930713, 16906...","[nan, 0.3657731398755853, nan, nan, nan]","[nan, 0.6214792467640008, nan, nan, nan]",NaN,10.125762,False
2,0.472328,-0.002487,-0.004660,-3.697688,2.707172,2.080631,0.182996,2.648589,-0.249116,OTRS:IN20:621,...,"[nan, 621.4792467640008, nan, nan, nan]",10.326044,"[1790.4556854108073, 1406.4239875995156, 1466....",3.128193,"[1690675174.3083148, 1690675175.7930713, 16906...","[nan, 0.3657731398755853, nan, nan, nan]","[nan, 0.6214792467640008, nan, nan, nan]",NaN,10.125762,False
2,0.472328,-0.002487,-0.004660,-3.697688,2.707172,2.080631,0.182996,2.648589,-0.249116,OTRS:IN20:621,...,"[nan, 621.4792467640008, nan, nan, nan]",3.28295,"[1790.4556854108073, 1406.4239875995156, 1466....",3.171362,"[1690675174.3083148, 1690675175.7930713, 16906...","[nan, 0.3657731398755853, nan

## Run optimization

In [16]:
n_step = 50
for _ in range(n_step):
    # optimization step
    t0 = time.time()
    X.step()
    print("Completed step {:d} ({:.2f} sec)".format(step, time.time() - t0))

CAPUT QUAD:IN20:121:BCTRL -0.0037404596619308
CAPUT QUAD:IN20:122:BCTRL -0.006053144205361604
CAPUT QUAD:IN20:361:BCTRL -3.688182035740465
CAPUT QUAD:IN20:371:BCTRL 2.5986414538919926
CAPUT QUAD:IN20:425:BCTRL 2.1171204317361116
CAPUT QUAD:IN20:441:BCTRL 1.2150069773197174
CAPUT QUAD:IN20:511:BCTRL 2.167915888130665
CAPUT QUAD:IN20:525:BCTRL 4.931116811931133
CAPUT SOLN:IN20:121:BCTRL 0.465212545786053
Completed step 0 (11.04 sec)
CAPUT QUAD:IN20:121:BCTRL 0.0030467202421277755
CAPUT QUAD:IN20:122:BCTRL -0.009482974838465452
CAPUT QUAD:IN20:361:BCTRL -3.6768051644787194
CAPUT QUAD:IN20:371:BCTRL 2.810719018906355
CAPUT QUAD:IN20:425:BCTRL -2.2858063727617264
CAPUT QUAD:IN20:441:BCTRL 0.8575505632907152
CAPUT QUAD:IN20:511:BCTRL 2.08644013479352
CAPUT QUAD:IN20:525:BCTRL 2.4860243778675795
CAPUT SOLN:IN20:121:BCTRL 0.47414092877414077
Completed step 1 (10.99 sec)
CAPUT QUAD:IN20:121:BCTRL -0.00805776353127216
CAPUT QUAD:IN20:122:BCTRL 0.00475549563501492
CAPUT QUAD:IN20:361:BCTRL -3.633

KeyboardInterrupt: 